In [1]:
import sys
import os
sys.path.append('/g/data/jk72/ll6859/ACCESS_aero_eval/')
from aercode import *
import matplotlib 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb
import cartopy.crs as ccrs
import seaborn as sns
import scipy.special
from datetime import datetime

import dask
import warnings
from dask.distributed import Client

In [2]:
plt.style.use('default')
warnings.filterwarnings('ignore')

# RVInvestigator

bx400

In [10]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/RVInvestigator/'
aer = pd.read_csv(f_path+'rvigaw_cn10_2016to2019_L2.csv', index_col='datetime')
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time']
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn10_avg','cn10_std','latitude','longitude']].dropna()
aer.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)


met = pd.read_csv(f_path+'RVI-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']>60]=np.nan
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2020_05_29d_RVI_1420_Radon.xlsx', sheet_name='RVI1420', index_col='DateTime')
rdn['time'] = pd.DatetimeIndex(rdn.index)
rdn.index = rdn['time']
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]','RnSTP']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std', 'RnSTP': 'radon_stp'}, inplace=True)

In [11]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)
obs = obs.assign(rdn)

In [15]:
# load in model data
rvi_bx400 = xr.open_dataset(f_path+'rvi1619_bx400_track.nc')

In [16]:
# merge model and obs
rvi_df = xr.merge([rvi_bx400,obs],compat='override')

# remove outliers
rvi_df = rvi_df.where(rvi_df['cn10_avg']<10000)
rvi_df = rvi_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
rvi_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2016-2019 RV Investigator voyage tracks'
rvi_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [5]:
rvi_df.load().to_netcdf(path=f_path+'rvi1619_bx400_df.nc')

cg283

In [ ]:
# load in model data
rvi_cg283 = xr.open_dataset(f_path+'rvi1619_cg283_track.nc')

In [ ]:
# merge model and obs
rvi_df = xr.merge([rvi_cg283,obs],compat='override')

# remove outliers
rvi_df = rvi_df.where(rvi_df['cn10_avg']<10000)
rvi_df = rvi_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
rvi_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016-2019 RV Investigator voyage tracks'
rvi_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
rvi_df.load().to_netcdf(path=f_path+'rvi1619_cg283_df.nc')

# Cold Water Trial

bx400

In [35]:
# f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/ColdWaterTrial/'

# df = pd.read_csv(f_path+'CWT_ProcessedAerosolData_1min.csv', delimiter=',', index_col='date')
# df['time'] = pd.DatetimeIndex(df.index)
# df.index = df['time'].astype('datetime64[ns]')
# df = df[['lat','lon','AirTemp_port','AirTemp_stbd','atmPress','WindSpdTru','Radon_hrly','CN_mean']]
# df = df.dropna()
# df = df.resample('1D', kind='time').mean()

# Reef 2 Rainforest

bx400

In [63]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Reef2Rain/'

df = pd.read_csv(f_path+'R2R_RV_aerosol.csv', index_col='date')
df['time'] = pd.DatetimeIndex(df.index)
df.index = df['time'].astype('datetime64[ns]')
df = df.resample('1D', kind='time').mean()
df = df.dropna()
df['air_temp'] = (df['AirTemp_port']+df['AirTemp_stbd'])/2
df['wind_spd'] = (df['WindSpdTru_port']+df['WindSpdTru_stbd'])/2
df = df[['lat','lon','air_temp','atmPress','WindDir', 'wind_spd', 'Radon_hrly','CN3_mean','CN10_mean','CCN_mean']]
df.rename(columns={'atmPress': 'air_pressure', 
                   'WindDir': 'wind_dir', 
                   'Radon_hrly': 'radon', 
                   'CN3_mean': 'cn3_avg', 
                   'CN10_mean': 'cn10_avg', 
                   'CCN_mean': 'ccn_avg'}, inplace=True)

In [ ]:
# convert observations to a dataset
obs = df.to_xarray()

In [ ]:
# load in model data
r2r_bx400 = xr.open_dataset(f_path+'r2r16_bx400_track.nc')

In [ ]:
# merge model and obs
r2r_df = xr.merge([r2r_bx400,obs],compat='override')

# remove outliers
r2r_df = r2r_df.where(r2r_df['cn10_avg']<10000)
r2r_df = r2r_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
r2r_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016 Reef 2 Rainforest voyage tracks'
r2r_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
r2r_df.load().to_netcdf(path=f_path+'r2r16_bx400_df.nc')

cg283

In [ ]:
# load in model data
r2r_cg283 = xr.open_dataset(f_path+'r2r16_cg283_track.nc')

In [ ]:
# merge model and obs
r2r_df = xr.merge([r2r_cg283,obs],compat='override')

# remove outliers
r2r_df = r2r_df.where(r2r_df['cn10_avg']<10000)
r2r_df = r2r_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
r2r_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016 Reef 2 Rainforest voyage tracks'
r2r_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
r2r_df.load().to_netcdf(path=f_path+'r2r16_cg283_df.nc')

# Ice 2 Equator

bx400

In [62]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Ice2Equator/'

df = pd.read_csv(f_path+'I2E_ProcessedAerosolData_1min.csv', index_col='date')
df['time'] = pd.DatetimeIndex(df.index)
df.index = df['time'].astype('datetime64[ns]')
df = df.resample('1D', kind='time').mean()
df = df.dropna()
df['air_temp'] = (df['AirTemp_port']+df['AirTemp_stbd'])/2
df['wind_spd'] = (df['WindSpdTru_port']+df['WindSpdTru_stbd'])/2
df['wind_dir'] = (df['WindDirTru_port']+df['WindDirTru_stbd'])/2
# df = df[['lat','lon','AirTemp_port','AirTemp_stbd','atmPress','Radon_hrly','CN_mean','ex_flag']]

,lat,lon,atmPress,WindDirRel_port,WindDirRel_stbd,WindDirRel_vmean,WindSpdRel_port,WindSpdRel_stbd,WindSpdTru_port,WindSpdTru_stbd,...,ex_flag,cont_flag,log_flag,aerosol_rh,aerosol_temperature,airchem_rh,airchem_temperature,smpsRH_calc,air_temp,wind_spd
time,,,,,,,,,,,,,,,,,,,,,
2016-04-26,-43.425598,147.798980,1016.026327,190.207946,196.635193,192.120014,10.277730,10.064156,14.409903,14.084389,...,1.000000,1.000000,0.922222,0.000000,0.000000,60.717929,20.801835,-28.334525,19.714112,14.247146
2016-04-27,-46.453184,150.204248,1005.839022,173.134326,189.197775,179.329456,13.999698,15.012399,22.193917,23.157896,...,1.000000,0.795139,0.609028,66.397992,16.370920,54.169172,21.469147,38.701443,15.434299,22.675906
2016-04-28,-50.156411,153.299807,1012.302944,78.071940,85.131612,81.816285,17.710548,18.634691,19.032362,20.902083,...,0.109028,0.000000,0.037500,57.819969,11.649275,35.298000,19.208571,25.455964,7.957166,19.967223
2016-04-29,-53.869242,156.549457,1011.410276,233.173366,241.993834,237.238586,21.865687,16.321303,25.630744,19.937445,...,0.143056,0.003472,0.130556,61.676514,9.258324,33.648926,18.399712,23.471443,5.162344,22.784095
2016-04-30,-57.302396,161.376896,995.362042,180.637695,184.035084,182.325656,10.012811,10.229069,20.324758,20.482928,...,1.000000,0.590278,0.196528,75.874296,8.014755,37.709409,18.652748,29.460880,4.395986,20.403843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-24,-0.725397,-169.285065,1006.969742,164.774518,196.074772,174.804574,17.438139,14.775604,21.283876,18.953288,...,0.265278,0.378472,0.009722,76.074568,27.366738,84.657693,25.467526,63.615640,27.786350,20.118582
2016-06-25,-3.954720,-172.005605,1006.584984,259.103907,261.435798,149.262433,9.201209,7.895921,15.955353,14.760302,...,0.545833,0.000000,0.034722,77.396107,28.345118,87.782147,26.080330,66.097677,28.908341,15.357827
2016-06-26,-7.430975,-174.879826,1006.550843,287.807962,288.569267,288.130286,14.384716,14.875127,15.526135,15.749447,...,0.000000,0.000000,0.009722,88.145260,27.085458,95.354336,25.644655,70.072667,28.196209,15.637791


In [ ]:
# convert observations to a dataset
obs = df.to_xarray()

In [ ]:
# load in model data
i2e_bx400 = xr.open_dataset(f_path+'i2e16_bx400_track.nc')

In [ ]:
# merge model and obs
i2e_df = xr.merge([i2e_bx400,obs],compat='override')

# remove outliers
i2e_df = i2e_df.where(i2e_df['cn10_avg']<10000)
i2e_df = i2e_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
i2e_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2016 Ice 2 Equator voyage tracks'
i2e_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
i2e_df.load().to_netcdf(path=f_path+'i2e16_bx400_df.nc')

cg283

In [ ]:
# load in model data
i2e_cg283 = xr.open_dataset(f_path+'i2e16_cg283_track.nc')

In [ ]:
# merge model and obs
i2e_df = xr.merge([i2e_cg283,obs],compat='override')

# remove outliers
i2e_df = i2e_df.where(i2e_df['cn10_avg']<10000)
i2e_df = i2e_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
i2e_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016 Ice 2 Equator voyage tracks'
i2e_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
i2e_df.load().to_netcdf(path=f_path+'i2e16_cg283_df.nc')

# CAPRICORN I

bx400

In [15]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Capricorn/'

aer = pd.read_hdf(f_path+'aerosol_1sec.h5')
aer.index.name = 'time'
aer = aer.resample('1D', kind='time').mean()

met = pd.read_csv(f_path+'in2016_v02uwy10sec.csv', parse_dates=[['date', 'time']], index_col='date_time')
met = met.resample('1D').mean()
met = met[['latitude(degree_north)','longitude(degree_east)','portAirTemp(degC)','stbdAirTemp(degC)','atmPressure(mbar)','rain(mm)']]
met = met.rename(columns = {"latitude(degree_north)": "lat", "longitude(degree_east)": "lon"})
met.index.name = 'time'

In [16]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)

<xarray.Dataset>
Dimensions:            (time: 36)
Coordinates:
  * time               (time) datetime64[ns] 2016-03-13 ... 2016-04-17
Data variables:
    ccn_0.5504         (time) float64 204.2 188.2 155.4 165.8 ... 201.2 nan nan
    ccn_sigma          (time) float64 15.0 14.32 13.04 13.55 ... 14.29 nan nan
    cn3                (time) float64 nan nan 7.617e+03 ... 320.1 458.1 nan
    cn3_sigma          (time) float64 nan nan 776.7 235.3 ... 36.48 50.4 nan
    lat                (time) float64 -42.89 -44.08 -46.49 ... -44.23 nan nan
    lon                (time) float64 147.4 146.4 142.8 142.0 ... 146.8 nan nan
    portAirTemp(degC)  (time) float64 15.28 12.83 11.36 13.69 ... 14.18 nan nan
    stbdAirTemp(degC)  (time) float64 15.27 12.85 11.52 13.7 ... 14.06 nan nan
    atmPressure(mbar)  (time) float64 1.02e+03 1.027e+03 1.027e+03 ... nan nan
    rain(mm)           (time) float64 0.04 0.003392 0.005318 ... 0.02776 nan nan

In [ ]:
# load in model data
cap1_bx400 = xr.open_dataset(f_path+'cap16_bx400_track.nc')

In [ ]:
# merge model and obs
cap1_df = xr.merge([cap1_bx400,obs],compat='override')

# remove outliers
cap1_df = cap1_df.where(cap1_df['cn10_avg']<10000)
cap1_df = cap1_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cap1_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2016 CAPRICORN I voyage tracks'
cap1_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
cap1_df.load().to_netcdf(path=f_path+'cap16_bx400_df.nc')

cg283

In [ ]:
# load in model data
cap1_cg283 = xr.open_dataset(f_path+'cap16_cg283_track.nc')

In [ ]:
# merge model and obs
cap1_df = xr.merge([cap1_cg283,obs],compat='override')

# remove outliers
cap1_df = cap1_df.where(cap1_df['cn10_avg']<10000)
cap1_df = cap1_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cap1_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016 CAPRICORN I voyage tracks'
cap1_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
cap1_df.load().to_netcdf(path=f_path+'cap16_cg283_df.nc')

# CAPRICORN II

bx400

In [18]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Capricorn/'

aer = pd.read_csv(f_path+'capricorn2_cpc3772_L2_1H.csv', index_col=0)
aer.index.name = 'time'
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time'].astype('datetime64[ns]')
aer = aer.resample('1D', kind='time').mean()

met = pd.read_csv(f_path+'in2018_v01uwy10sec.csv', parse_dates=[['date', 'time']], index_col='date_time')
met = met[['latitude(degree_north)','longitude(degree_east)']]
met = met.rename(columns = {"latitude(degree_north)": "lat", "longitude(degree_east)": "lon"})
met.index.name = 'time'
met['lat'] = pd.to_numeric(met['lat'], errors='coerce')
met['lon'] = pd.to_numeric(met['lon'], errors='coerce')
met = met.resample('1D').mean()

In [19]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)

<xarray.Dataset>
Dimensions:       (time: 43)
Coordinates:
  * time          (time) datetime64[ns] 2018-01-10 2018-01-11 ... 2018-02-21
Data variables:
    cn_med        (time) float64 nan 874.3 1.028e+03 750.2 ... 204.2 290.1 407.8
    cn_mad        (time) float64 nan 5.45 6.213 4.767 ... 10.26 3.715 4.773 3.4
    cn_avg        (time) float64 nan 874.0 1.028e+03 750.6 ... 203.8 290.7 407.4
    cn_std        (time) float64 nan 9.021 11.26 8.591 ... 5.472 7.178 9.241
    cn_count      (time) float64 0.0 2.208 3.583 ... 3.564e+03 3.596e+03 149.5
    cn_sigma_med  (time) float64 nan 16.18 17.36 15.44 ... 0.2485 0.2955 10.31
    cn_sigma_avg  (time) float64 nan 16.65 18.24 16.09 ... 0.2584 0.31 0.3692
    lat           (time) float64 -43.12 -44.05 -45.15 ... -52.59 -47.97 -43.95
    lon           (time) float64 147.4 146.5 145.9 146.5 ... 143.2 144.4 146.9

In [ ]:
# load in model data
cap2_bx400 = xr.open_dataset(f_path+'cap18_bx400_track.nc')

In [ ]:
# merge model and obs
cap2_df = xr.merge([cap2_bx400,obs],compat='override')

# remove outliers
cap2_df = cap2_df.where(cap2_df['cn10_avg']<10000)
cap2_df = cap2_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cap2_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2018 CAPRICORN II voyage tracks'
cap2_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
cap2_df.load().to_netcdf(path=f_path+'cap18_bx400_df.nc')

cg283

In [ ]:
# load in model data
cap2_cg283 = xr.open_dataset(f_path+'cap18_cg283_track.nc')

In [ ]:
# merge model and obs
cap2_df = xr.merge([cap2_cg283,obs],compat='override')

# remove outliers
cap2_df = cap2_df.where(cap2_df['cn10_avg']<10000)
cap2_df = cap2_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cap2_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2018 CAPRICORN II voyage tracks'
cap2_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
cap2_df.load().to_netcdf(path=f_path+'cap18_cg283_df.nc')

# MARCUS

bx400

In [21]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/AuroraAustralis/'

aer = pd.read_csv(f_path+'CN_MARCUS_1hr.csv', index_col=0)
aer.index.name = 'time'
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time']
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn_avg','cn_std','lat','lon']].dropna()
aer.rename(columns={'cn_avg': 'cn10_avg', 'cn_std': 'cn10_std'}, inplace=True)

met = pd.read_csv(f_path+'marcus-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

In [22]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)

<xarray.Dataset>
Dimensions:       (time: 78)
Coordinates:
  * time          (time) datetime64[ns] 2017-10-29 2017-10-30 ... 2018-03-24
Data variables:
    cn10_avg      (time) float64 816.1 490.0 380.4 337.7 ... 173.8 145.9 251.7
    cn10_std      (time) float64 56.36 23.12 18.13 23.07 ... 17.83 14.46 13.57
    lat           (time) float64 -43.22 -43.38 -44.07 ... -53.2 -48.83 -44.76
    lon           (time) float64 147.4 147.3 144.4 139.9 ... 157.6 152.9 148.7
    air_temp      (time) float64 nan nan 11.61 9.694 ... 5.923 7.847 11.07 14.9
    air_pressure  (time) float64 nan nan 1.012e+03 ... 1.02e+03 1.001e+03
    total_rain    (time) float64 nan nan 1.9 0.4 3.2 2.9 ... 1.5 0.0 0.0 0.0 0.0
    wind_dir      (time) float64 nan nan 301.4 272.5 ... 284.1 274.6 64.34 223.3
    wind_spd      (time) float64 nan nan 10.26 9.215 ... 13.0 6.305 10.34 4.631

In [ ]:
# load in model data
aa_bx400 = xr.open_dataset(f_path+'aa1718_bx400_track.nc')

In [ ]:
# merge model and obs
aa_df = xr.merge([aa_bx400,obs],compat='override')

# remove outliers
aa_df = aa_df.where(aa_df['cn10_avg']<10000)
aa_df = aa_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
aa_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2017-2018 MARCUS voyage tracks'
aa_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
aa_df.load().to_netcdf(path=f_path+'aa1718_bx400_df.nc')

cg283

In [ ]:
# load in model data
aa_cg283 = xr.open_dataset(f_path+'aa1718_cg283_track.nc')

In [ ]:
# merge model and obs
aa_df = xr.merge([aa1718_cg283,obs],compat='override')

# remove outliers
aa_df = aa_df.where(aa_df['cn10_avg']<10000)
aa_df = aa_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
aa_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2017-2018 MARCUS voyage tracks'
aa_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
aa_df.load().to_netcdf(path=f_path+'aa1718_cg283_df.nc')

# TAN1802

bx400

In [24]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/TAN1802/'
aer = xr.open_dataset(f_path+'CPC3010_TAN1802_2018_level_01.nc')
aer = aer.rename({'number_concentration_of_ambient_aerosol_particles_in_air':'cn10',
                  'number_concentration_of_ambient_aerosol_particles_in_air_flag':'cn10_flag'})
aer = aer.where(aer.cn10_flag == 1)
aer['cn10'] = aer['cn10']/1000000
aer = aer.resample(time='1D').mean()
aer = aer.rename({'cn10': 'cn10_avg'})

uw = keycutter(f_path+'weather.csv','Time (UTC)', 'Latitude', 'Longitude')


met = pd.read_csv(f_path+'TAN-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

In [25]:
# convert observations to a dataset
obs = aer
obs = obs.assign(uw)
obs = obs.assign(met)

<xarray.Dataset>
Dimensions:       (time: 42)
Coordinates:
  * time          (time) datetime64[ns] 2018-02-09 2018-02-10 ... 2018-03-22
Data variables:
    cn10_avg      (time) float64 422.0 912.9 977.9 98.8 ... nan nan nan nan
    cn10_flag     (time) float32 1.0 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan nan
    lat           (time) float64 nan -46.63 -52.12 -55.81 ... nan nan nan nan
    lon           (time) float64 nan 174.6 174.4 174.3 174.0 ... nan nan nan nan
    air_temp      (time) float64 nan nan 11.1 8.365 5.766 ... nan nan nan nan
    air_pressure  (time) float64 nan nan 1e+03 990.2 ... nan nan nan nan
    total_rain    (time) float64 nan nan 70.25 28.76 5.56 ... nan nan nan nan
    wind_dir      (time) float64 nan nan 63.31 140.0 197.6 ... nan nan nan nan
    wind_spd      (time) float64 nan nan 12.52 11.73 8.407 ... nan nan nan nan

In [ ]:
# load in model data
tan_bx400 = xr.open_dataset(f_path+'tan1718_bx400_track.nc')

In [ ]:
# merge model and obs
tan_df = xr.merge([tan_bx400,obs],compat='override')

# remove outliers
tan_df = tan_df.where(tan_df['cn10_avg']<10000)
tan_df = tan_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
tan_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2017-2018 TAN1802 voyage tracks'
tan_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
tan_df.load().to_netcdf(path=f_path+'tan1718_bx400_df.nc')

cg283

In [ ]:
# load in model data
tan_cg283 = xr.open_dataset(f_path+'tan1718_cg283_track.nc')

In [ ]:
# merge model and obs
tan_df = xr.merge([tan_cg283,obs],compat='override')

# remove outliers
tan_df = tan_df.where(tan_df['cn10_avg']<10000)
tan_df = tan_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
tan_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2017-2018 TAN1802 voyage tracks'
tan_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
tan1718_df.load().to_netcdf(path=f_path+'tan1718_cg283_df.nc')

# Cape Grim

bx400

In [27]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/CGO/'
aer = pd.read_excel(f_path+'cgbpaps_CN  2016-2018.xlsx', index_col='date')
aer = aer.astype(np.float64).resample('1D',kind='time').mean()
aer.index.name = 'time'
aer = aer[['cn3772_mean','cn3772_sd']]
aer.rename(columns={'cn3772_mean': 'cn10_avg', 'cn3772_sd': 'cn10_std'}, inplace=True)

met = pd.read_excel(f_path+'CG_met_2015-2018.xlsx', index_col='date')
met = met[['TempGC','Pressure','Rain','VWS10m','VWD10m']]
met.index.name = 'time'
met = met.resample('1D', kind='time').mean()
met.rename(columns={'TempGC': 'air_temp', 'Rain': 'total_rain', 'Pressure': 'air_pressure', 'VWD10m': 'wind_dir', 'VWS10m': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2021_08_25a_CGO_Radon_2015-2018_ForLiam.xlsx', sheet_name='2015-2018-RadonMetCO2', index_col='DateTime')
rdn.index.name = 'time'
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std'}, inplace=True)

In [28]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)
obs = obs.assign(rdn)

<xarray.Dataset>
Dimensions:       (time: 1096)
Coordinates:
  * time          (time) datetime64[ns] 2016-01-01 2016-01-02 ... 2018-12-31
Data variables:
    cn10_avg      (time) float64 1.763e+03 1.063e+03 ... 1.013e+03 673.8
    cn10_std      (time) float64 74.55 403.7 439.3 ... 56.63 36.09 16.5
    air_temp      (time) float64 17.17 16.47 17.46 17.12 ... 16.3 15.09 15.99
    air_pressure  (time) float64 1.002e+03 1.007e+03 ... 1.003e+03 1.001e+03
    total_rain    (time) float64 0.0 0.0 0.0 0.0 0.025 ... 0.008333 0.05 0.0 0.0
    wind_spd      (time) float64 12.57 7.175 9.167 7.817 ... 8.803 8.612 7.688
    wind_dir      (time) float64 193.2 154.7 87.83 93.99 ... 189.4 209.9 223.6
    radon         (time) float64 939.3 509.1 404.2 504.2 ... 171.8 114.4 111.7
    radon_std     (time) float64 28.83 23.38 21.67 22.5 ... 13.67 12.43 12.4

In [ ]:
# load in model data
cgo_bx400 = xr.open_dataset(f_path+'cgo1718_bx400_track.nc')

In [ ]:
# merge model and obs
cgo_df = xr.merge([cgo_bx400,obs],compat='override')

# remove outliers
cgo_df = cgo_df.where(cgo_df['cn10_avg']<10000)
cgo_df = cgo_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cgo_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations at the 2016-2018 Cape Grim Baseline Air Pollution Station'
cgo_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
cgo_df.load().to_netcdf(path=f_path+'cgo1718_bx400_df.nc')

cg283

In [ ]:
# load in model data
cgo_cg283 = xr.open_dataset(f_path+'cgo1718_cg283_track.nc')

In [ ]:
# merge model and obs
cgo_df = xr.merge([cgo_cg283,obs],compat='override')

# remove outliers
cgo_df = cgo_df.where(cgo_df['cn10_avg']<10000)
cgo_df = cgo_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cgo_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations at the 2016-2018 Cape Grim Baseline Air Pollution Station'
cgo_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
cgo1618_df.load().to_netcdf(path=f_path+'cgo1618_cg283_df.nc')

# Macquarie Island

bx400

In [30]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/MQI/'
aer = pd.read_hdf(f_path+'ACRE_CN10_1H_FINAL.h5')
aer.index.name = 'time'
aer = aer[['cn_avg','cn_std']]
aer = aer.resample('1D', kind='time').mean()
aer.rename(columns={'cn_avg': 'cn10_avg', 'cn_std': 'cn10_std'}, inplace=True)

met = []
all_files = gb.glob(f_path+'AWS_hourly/*.csv')
met = pd.concat([pd.read_csv(v) for v in all_files])
met.index = met['MIN_OBSERVATION_DATE_'].astype('datetime64[ns]')
met.index.name = 'time'
met['AVG_RAINFALL'] = met['AVG_RAINFALL'].diff()
met['AVG_RAINFALL'][met['AVG_RAINFALL']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['AVG_RAINFALL'] = met_tmp['AVG_RAINFALL'].resample('1D', kind='time').sum()
met = met[['AVG_AIR_TEMPERATURE','AVG_WIND_DIRECTION','AVG_WIND_SPEED','AVG_AIR_PRESSURE','AVG_RAINFALL']]
met.rename(columns={'AVG_AIR_TEMPERATURE': 'air_temp', 'AVG_RAINFALL': 'total_rain', 'AVG_AIR_PRESSURE': 'air_pressure', 'AVG_WIND_DIRECTION': 'wind_dir', 'AVG_WIND_SPEED': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2021_09_15a_MQI_Radon_2013-2019_ForLiamLamprey.xlsx',
                    sheet_name='Radon_MQI_2013-2019',
                    index_col='DateTime')
rdn.index.name = 'time'
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std'}, inplace=True)

In [31]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)
obs = obs.assign(rdn)

<xarray.Dataset>
Dimensions:       (time: 700)
Coordinates:
  * time          (time) datetime64[ns] 2016-04-05 2016-04-06 ... 2018-03-05
Data variables:
    cn10_avg      (time) float64 nan 246.9 304.2 228.5 ... 300.1 236.5 293.0
    cn10_std      (time) float64 nan 12.31 9.519 16.07 ... 19.85 15.71 4.867
    air_temp      (time) float64 7.654 4.238 6.451 5.62 ... 7.989 6.528 6.124
    wind_dir      (time) float64 340.5 292.3 332.9 317.4 ... 295.5 296.4 278.1
    wind_spd      (time) float64 32.44 19.81 28.27 26.49 ... 11.3 19.87 18.3
    air_pressure  (time) float64 982.9 993.9 992.7 ... 999.6 990.4 1.002e+03
    total_rain    (time) float64 3.2 2.0 0.2 6.4 7.2 ... 0.4 0.2 17.61 12.99 5.4
    radon         (time) float64 154.2 191.8 75.12 93.58 ... 71.29 82.54 55.0
    radon_std     (time) float64 47.12 48.71 41.12 42.88 ... 37.75 38.62 36.21

In [ ]:
# load in model data
mqi_bx400 = xr.open_dataset(f_path+'mqi1718_bx400_track.nc')

In [ ]:
# merge model and obs
mqi_df = xr.merge([mqi_bx400,obs],compat='override')

# remove outliers
mqi_df = mqi_df.where(mqi_df['cn10_avg']<10000)
mqi_df = mqi_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
mqi_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations at the 2016-2018 Macquarie Island Research Station'
mqi_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
mqi_df.load().to_netcdf(path=f_path+'mqi1618_bx400_df.nc')

cg283

In [ ]:
# load in model data
mqi_cg283 = xr.open_dataset(f_path+'mqi1718_cg283_track.nc')

In [ ]:
# merge model and obs
mqi_df = xr.merge([mqi1618_cg283,obs],compat='override')

# remove outliers
mqi_df = mqi_df.where(mqi_df['cn10_avg']<10000)
mqi_df = mqi_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
mqi_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations at the 2016-2018 Macquarie Island Research Station'
mqi_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [ ]:
mqi_df.load().to_netcdf(path=f_path+'mqi1618_cg283_df.nc')